<a href="https://colab.research.google.com/github/parus-cristatus/tolokapizza/blob/main/tolokakit/Crop_images_by_bounding_boxes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Template used for the snippet: http://bit.ly/3EqCDtQ
Multiple objects can be outlined

Sample image links:

https://tolokaadmin.s3.yandex.net/demo/datasets/ed178eb2-31e1-4da9-81f0-3c363b11ba5a

https://tolokaadmin.s3.yandex.net/demo/datasets/f4ec9a85-e2e2-4bad-acd9-adde40f07b9c

https://tolokaadmin.s3.yandex.net/demo/datasets/c80c3145-03e7-4030-8c67-0772a8056d42

Cropped images are matched with original image by using part of its ID

In [ ]:
!pip install toloka-kit

In [2]:
import toloka.client as toloka
import requests
from PIL import Image

In [ ]:
target = 'SANDBOX' #PRODUCTION
toloka_client = toloka.TolokaClient(input("Enter your token:"), target)
print(toloka_client.get_requester())

In [4]:
POOL = '1335298' # Pool ID to get accepted assignments from

In [5]:
# get required data to manipulate

assignments = list(toloka_client.get_assignments(pool_id=POOL, status='ACCEPTED')) # assignments must be in accepted status to crop
tasks = [assignment.tasks for assignment in assignments]
images = [img.input_values['image'] for task in tasks for img in task]
bboxes = [sol for assignment in assignments for sol in assignment.solutions]

In [ ]:
for idx, img in enumerate(images):
    # print(img)
    _id = img.split('-')[-1]
    
    # create 'imgs' folder for initial images and 'crops' folder for cropped images

    with open(f'imgs/pic_{idx}.jpg', 'wb') as handle:
        response = requests.get(img, stream=True)
        handle.write(response.content)
        im = Image.open(f'imgs/pic_{idx}.jpg')
        nat_width, nat_height = im.size

    for img_idx, coord in enumerate((x for x in bboxes[idx].output_values['result'])):
        coords = (
            float(coord.get('left')), 
            float(coord.get('top')), 
            float(coord.get('width')), 
            float(coord.get('height'))
            )

        left = coords[0] * nat_width # result.left
        top = coords[1] * nat_height  # result.top

        right = left + nat_width * coords[2]  # result.width
        bottom = top + nat_height * coords[3]  # result.height

        # print(left, top, right, bottom)

        im1 = im.crop((left, top, right, bottom))
        im1.save(f'crops/{_id}_{img_idx}.jpg')